In [1]:
from ultralytics import YOLO
import os

project = os.path.join("..","..","runs")
yolo_models = ["yolov8s", "yolov8m"] # select yolov8s and yolov8m for further hyperparamter search and finetune

In [2]:
for model_name in yolo_models:
    print(f"Model : {model_name}")
    model = YOLO(model_name)
    model.tune(data="data.yaml", epochs=30, iterations=100, optimizer="AdamW", plots=False, save=False, val=False,name=f'{model_name}_hyp_finetune',project=project)

Model : yolov8s
Tuner: Initialized Tuner instance with 'tune_dir=..\..\runs\tune8'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/100 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
Saved ..\..\runs\tune8\tune_scatter_plots.png
Saved ..\..\runs\tune8\tune_fitness.png

Tuner: 1/100 iterations complete  (124.36s)
Tuner: Results saved to ..\..\runs\tune8
Tuner: Best fitness=0.4057 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.77781, 'metrics/recall(B)': 0.59375, 'metrics/mAP50(B)': 0.69483, 'metrics/mAP50-95(B)': 0.37357, 'val/box_loss': 1.55685, 'v

In [28]:
import pandas as pd
df = pd.read_csv(os.path.join(project,"yolov8s_tune","tune_results.csv"))
df.sort_values(by="fitness",ascending=False)



,fitness,lr0,lrf,momentum,weight_decay,warmup_epochs,warmup_momentum,box,cls,dfl,...,translate,scale,shear,perspective,flipud,fliplr,bgr,mosaic,mixup,copy_paste
82,0.53030,0.00520,0.01373,0.78307,0.00047,1.83134,0.57311,5.55754,0.29102,2.15464,...,0.12186,0.27246,0.0,0.0,0.0,0.34646,0.0,0.98814,0.0,0.0
75,0.52844,0.00469,0.01402,0.81704,0.00047,1.64718,0.67259,5.88582,0.30737,2.28370,...,0.10950,0.32015,0.0,0.0,0.0,0.35439,0.0,0.88960,0.0,0.0
69,0.52352,0.00463,0.01373,0.82307,0.00047,1.64718,0.67259,5.42996,0.29007,2.15464,...,0.11490,0.29982,0.0,0.0,0.0,0.34263,0.0,0.94916,0.0,0.0
68,0.52328,0.00431,0.01079,0.87811,0.00046,1.72258,0.69575,5.51495,0.27681,2.08879,...,0.09168,0.33485,0.0,0.0,0.0,0.33200,0.0,0.96171,0.0,0.0
96,0.52253,0.00583,0.01299,0.76951,0.00053,1.32911,0.57311,5.68494,0.25837,2.15464,...,0.12186,0.22731,0.0,0.0,0.0,0.28795,0.0,0.94049,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,0.39078,0.01001,0.01299,0.91203,0.00044,3.59326,0.84851,6.93812,0.43559,1.81896,...,0.08128,0.50032,0.0,0.0,0.0,0.55468,0.0,1.00000,0.0,0.0
3,0.38501,0.01121,0.00960,0.94515,0.00040,2.63028,0.63501,7.04875,0.50795,1.47980,...,0.09993,0.49211,0.0,0.0,0.0,0.49525,0.0,1.00000,0.0,0.0
5,0.37238,0.01019,0.01101,0.93700,0.00047,2.91529,0.83398,8.41079,0.42004,1.73003,...,0.08678,0.50000,0.0,0.0,0.0,0.50000,0.0,1.00000,0.0,0.0
1,0.36532,0.01000,0.01026,0.94393,0.00050,3.00000,0.79008,6.91915,0.50795,1.50000,...,0.10375,0.47895,0.0,0.0,0.0,0.49525,0.0,1.00000,0.0,0.0


In [29]:
import yaml

model_name = "yolov8s"
print(f"Model : {model_name}")
model = YOLO(model_name)
best_params_filepath = os.path.join(project,"yolov8s_tune","best_hyperparameters.yaml")
with open(best_params_filepath, 'r') as file:
    best_hyperparameters = yaml.safe_load(file)

results = model.train(data="data.yaml", epochs=200, imgsz=640, optimizer="AdamW",name=f'{model_name}_hyp_finetune_best_param',project=project,**best_hyperparameters)



Model : yolov8s
Ultralytics YOLOv8.2.93  Python-3.10.6 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=..\..\runs, name=yolov8s_hyp_finetune_best_param, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

train: Scanning C:\Users\i_kaz\datasets\StreetOwl\train\labels.cache... 72 images, 0 backgrounds, 0 corrupt: 100%|██████████| 72/72 [00:00<?, ?it/s]
val: Scanning C:\Users\i_kaz\datasets\StreetOwl\valid\labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]


Plotting labels to ..\..\runs\yolov8s_hyp_finetune_best_param\labels.jpg... 
optimizer: AdamW(lr=0.0052, momentum=0.78307) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00047), 63 bias(decay=0.0)


2024/09/15 09:26:10 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.


MLflow: logging run_id(a616103e2cd94049bf4bbd44907477e5) to runs\mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs\mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to ..\..\runs\yolov8s_hyp_finetune_best_param
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    200/200      4.62G      0.408     0.2094      1.168         88        640: 100%|██████████| 5/5 [00:00<00:00,  5.06it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  8.26it/s]

                   all         21        192      0.913      0.703      0.831       0.49



200 epochs completed in 0.150 hours.
Optimizer stripped from ..\..\runs\yolov8s_hyp_finetune_best_param\weights\last.pt, 22.5MB
Optimizer stripped from ..\..\runs\yolov8s_hyp_finetune_best_param\weights\best.pt, 22.5MB

Validating ..\..\runs\yolov8s_hyp_finetune_best_param\weights\best.pt...
Ultralytics YOLOv8.2.93  Python-3.10.6 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.10it/s]


                   all         21        192      0.877      0.745      0.829      0.505
Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to ..\..\runs\yolov8s_hyp_finetune_best_param
MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow=False'


In [30]:

df = pd.read_csv(os.path.join(project,"yolov8m_tune","tune_results.csv"))
df.sort_values(by="fitness",ascending=False)




,fitness,lr0,lrf,momentum,weight_decay,warmup_epochs,warmup_momentum,box,cls,dfl,...,translate,scale,shear,perspective,flipud,fliplr,bgr,mosaic,mixup,copy_paste
94,0.52639,0.00309,0.00985,0.78255,0.00034,2.32036,0.73596,3.77458,0.33859,1.21590,...,0.04441,0.47475,0.0,0.0,0.0,0.09327,0.0,0.55585,0.0,0.0
86,0.52374,0.00455,0.01000,0.74231,0.00034,2.28255,0.84902,4.37289,0.36164,1.48249,...,0.04441,0.47475,0.0,0.0,0.0,0.09600,0.0,0.64538,0.0,0.0
92,0.51692,0.00352,0.00770,0.78286,0.00036,2.28255,0.78766,3.96794,0.46106,1.77714,...,0.03227,0.29526,0.0,0.0,0.0,0.08947,0.0,0.52405,0.0,0.0
83,0.50810,0.00455,0.00927,0.74682,0.00038,2.28255,0.81080,4.81845,0.40627,1.42391,...,0.04257,0.49035,0.0,0.0,0.0,0.09788,0.0,0.65655,0.0,0.0
98,0.50520,0.00322,0.00686,0.78246,0.00035,2.44112,0.80060,3.96794,0.46877,1.61274,...,0.03227,0.31124,0.0,0.0,0.0,0.08445,0.0,0.54528,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,0.23322,0.00991,0.01000,0.93700,0.00053,3.19103,0.91058,6.99125,0.50000,1.21696,...,0.08922,0.54024,0.0,0.0,0.0,0.52438,0.0,0.99158,0.0,0.0
1,0.17820,0.01060,0.01000,0.93700,0.00046,3.00000,0.80000,7.50000,0.51217,1.55107,...,0.09549,0.57368,0.0,0.0,0.0,0.52095,0.0,1.00000,0.0,0.0
13,0.16887,0.00964,0.01227,0.92395,0.00034,3.23091,0.92526,5.90349,0.49507,1.92171,...,0.04283,0.82730,0.0,0.0,0.0,0.61949,0.0,0.99656,0.0,0.0
10,0.05101,0.01330,0.00888,0.96315,0.00046,3.54609,0.80000,8.94983,0.50000,1.68469,...,0.12514,0.77756,0.0,0.0,0.0,0.32297,0.0,1.00000,0.0,0.0


In [31]:
import yaml

model_name = "yolov8m"
print(f"Model : {model_name}")
model = YOLO(model_name)
best_params_filepath = os.path.join(project,"yolov8m_tune","best_hyperparameters.yaml")
with open(best_params_filepath, 'r') as file:
    best_hyperparameters = yaml.safe_load(file)

results = model.train(data="data.yaml", epochs=200, imgsz=640, optimizer="AdamW",name=f'{model_name}_hyp_finetune_best_param',project=project,**best_hyperparameters)



Model : yolov8m
Ultralytics YOLOv8.2.93  Python-3.10.6 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=..\..\runs, name=yolov8m_hyp_finetune_best_param, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

train: Scanning C:\Users\i_kaz\datasets\StreetOwl\train\labels.cache... 72 images, 0 backgrounds, 0 corrupt: 100%|██████████| 72/72 [00:00<?, ?it/s]
val: Scanning C:\Users\i_kaz\datasets\StreetOwl\valid\labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]


Plotting labels to ..\..\runs\yolov8m_hyp_finetune_best_param\labels.jpg... 
optimizer: AdamW(lr=0.00309, momentum=0.78255) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.00034), 83 bias(decay=0.0)


2024/09/15 09:35:45 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.


MLflow: logging run_id(edcabeb9a393401c9fb309cdc847d3ad) to runs\mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs\mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to ..\..\runs\yolov8m_hyp_finetune_best_param
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    200/200      7.41G     0.2417     0.2052     0.6439         90        640: 100%|██████████| 5/5 [00:02<00:00,  2.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.68it/s]

                   all         21        192      0.861       0.74      0.843      0.534



200 epochs completed in 0.269 hours.
Optimizer stripped from ..\..\runs\yolov8m_hyp_finetune_best_param\weights\last.pt, 52.0MB
Optimizer stripped from ..\..\runs\yolov8m_hyp_finetune_best_param\weights\best.pt, 52.0MB

Validating ..\..\runs\yolov8m_hyp_finetune_best_param\weights\best.pt...
Ultralytics YOLOv8.2.93  Python-3.10.6 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.34it/s]


                   all         21        192      0.861       0.74      0.843      0.533
Speed: 0.2ms preprocess, 8.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to ..\..\runs\yolov8m_hyp_finetune_best_param
MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow=False'
